In [1]:
import os
import re
import json
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from concurrent.futures import ThreadPoolExecutor
from spectral import open_image

In [ ]:
np.random.seed(42)
random.seed(42)

In [ ]:
def spectral_angle_mapper(image, reference_spectrum, threshold=0.15):
    """
    Compute a binary mask using Spectral Angle Mapper (SAM).

    Args:
        image (np.ndarray): Hyperspectral cube (H x W x Bands)
        reference_spectrum (np.ndarray): Reference pixel spectrum
        threshold (float): Angle threshold for classification (radians)
    Returns:
        np.ndarray: Binary mask (H x W)
    """
    ref_norm = reference_spectrum / np.linalg.norm(reference_spectrum)
    pixels = image.reshape(-1, image.shape[2])
    pixel_norms = np.linalg.norm(pixels, axis=1)
    pixel_norms[pixel_norms == 0] = 1e-6
    dot_product = np.dot(pixels, ref_norm)
    cos_angles = np.clip(dot_product / (pixel_norms * np.linalg.norm(ref_norm)), -1.0, 1.0)
    angles = np.arccos(cos_angles)
    return (angles.reshape(image.shape[0], image.shape[1]) < threshold).astype(np.uint8)


def crop_to_mask(image, mask, margin=5):
    """
    Crop the hyperspectral image based on the binary mask.

    Returns:
        cropped_image, cropped_mask, (y_min, x_min)
    """
    y_indices, x_indices = np.where(mask == 1)
    if len(y_indices) == 0 or len(x_indices) == 0:
        return None, None, None

    y_min, y_max = max(y_indices.min() - margin, 0), min(y_indices.max() + margin, image.shape[0])
    x_min, x_max = max(x_indices.min() - margin, 0), min(x_indices.max() + margin, image.shape[1])

    return (
        image[y_min:y_max, x_min:x_max, :],
        mask[y_min:y_max, x_min:x_max],
        (y_min, x_min),
    )

In [ ]:
def process_and_save_npz(hdr_path, input_root, output_root, ref_yx=(255, 247), threshold=0.19):
    """
    Process a hyperspectral HDR image: generate mask, crop jam region, and save compressed NPZ.
    """
    try:
        img = open_image(hdr_path)
        cube = img.load()

        ref_pixel = cube[ref_yx[0], ref_yx[1], :].reshape(-1)
        mask = spectral_angle_mapper(cube, ref_pixel, threshold)
        cropped_cube, cropped_mask, offset = crop_to_mask(cube, mask)

        if cropped_cube is None:
            print(f"Skipped (no jam was detected): {hdr_path}")
            return

        rel_path = os.path.relpath(hdr_path, start=input_root)
        rel_dir = os.path.dirname(rel_path)
        base_id = re.search(r"REFLECTANCE_(\\d+)", hdr_path).group(1)

        save_dir = os.path.join(output_root, rel_dir)
        os.makedirs(save_dir, exist_ok=True)
        save_path = os.path.join(save_dir, f"cropped_{base_id}.npz")

        metadata = {
            "shape": cropped_cube.shape,
            "dtype": str(cropped_cube.dtype),
            "bands": cropped_cube.shape[2],
            "source": hdr_path,
        }

        np.savez_compressed(save_path, cube=cropped_cube, offset=np.array(offset), metadata=json.dumps(metadata))
        print(f"Saved: {save_path}")

    except Exception as e:
        print(f"Error processing {hdr_path}: {e}")


In [ ]:
input_root = "path/to/input"
output_root = "path/to/output"
paths_txt = "path/to/paths.txt"

with open(paths_txt, "r", encoding="utf-8") as f:
    hdr_paths = [os.path.join(input_root, line.strip()) for line in f if line.strip().endswith(".hdr")]

def safe_process(path):
    try:
        return process_and_save_npz(path, input_root, output_root)
    except Exception as e:
        return f"Error: {path} — {e}"

# Parallel execution
with ThreadPoolExecutor(max_workers=4) as executor:
    for i, result in enumerate(executor.map(safe_process, hdr_paths)):
        if i % 50 == 0:
            print(f"[{i}/{len(hdr_paths)}] {result}")
